# Running Semantic Functions Inline


* Let's dynamically generate the prompt using complex rules at runtime
* Writing prompts by editing C# code instead of TXT files. 
* Easily creating demos, like this document

For now we'll use only the `{{$input}}` variable, and see more complex templates later.


Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.5.0"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.5.0

create a semantic function used to summarize content, be a pirate:

In [2]:
string skPrompt = """
{{$input}}

Summarize the content above in 3 sentences. Sound like a drunk pirate!
""";

Let's configure the prompt, e.g. allowing for some creativity and a sufficient number of tokens.

In [3]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.2,
    TopP = 0.5
};

In [4]:
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);

var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);

var renderedPrompt = await promptTemplate.RenderAsync(kernel);

Console.WriteLine(renderedPrompt);



Summarize the content above in 3 sentences. Sound like a drunk pirate!


Let's transform the prompt template into a function

In [5]:
var summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Set up some content to summarize, taken from [History of Famous Cocktails](https://thepourhousempls.com/blog/history-of-famous-cocktails/).

In [6]:
//https://thepourhousempls.com/blog/history-of-famous-cocktails/

var input = """
There is more to a cocktail than just a tiny paper umbrella or a twist of an orange peel. If you think that a cocktail is simply soda or juice added to a shot of vodka or rum, let’s expand our knowledge and get a little more technical. 

There are so many caveats when it comes to cocktail making and mixology, but where did these unique drinks originate from? What exactly makes a cocktail, a cocktail? Well, a couple of things like spirits, bitters, water, and sugar, but also the history of cocktails.

THE BRIEF HISTORY OF COCKTAILS
While considered an American concoction, cocktails were actually inspired by British punches back in the day. British punches were essentially big bowls of different spirits mixed together with spices, fruit juice, and other additions, made popular in the 18th century. 

The actual word “cocktail” was seen in the 1700s among British newspapers but wasn’t truly defined until 1806 when it appeared in The Balance and Columbian Repository of Hudson, New York. The publication nailed down what we know today as a cocktail: “a stimulating liquor composed of any kind of sugar, water, and bitters,” providing the origin of the word cocktail.

Now the person who took cocktail making, or bartending, to the next level was Jerry Thomas. Jerry, also known as the “Professor,” was born in Connecticut and is one of the most prolific American bartenders. 

While he didn’t make the first cocktail in the world, he definitely knew what he was doing. He worked all across America and Europe, writing one of the very first inclusive recipe books on cocktails. The book was called “The Bartender’s Guide,” pretty much an encyclopedia of all mixed drinks that later became a standard for bartenders.

The more travel and industrialization that happened, the more opportunities that were made in the cocktail industry. One big innovation that occurred really changed the cocktail-making game in the history of cocktails, and that was ice. 

Can you imagine going to a bar today and ordering your favorite cocktail only for it to show up at room temperature? Luckily for Frederic Tudor, we don’t have to. Frederic, “The Ice King” Tudor, was the mastermind behind figuring out how to transport ice and maintain it at a colder climate within warmer ports. 

The venture eventually succeeded and made him a billionaire. The opportunities and possibilities of cocktail making grew tremendously as more ice was now available within the United States and internationally.

THE RISE AND FALL OF COCKTAIL POPULARITY
After a rise in the cocktail craft, a little thing called Prohibition happened and sadly put a stop on cocktail consumption. After the Prohibition laws were repealed, many of the states’ most talented bartenders had moved away to continue their craft abroad. 

In a way, the World Wars actually helped spark cocktail culture again as it brought exposure to different cultures and different liquors like the Polynesian rum.

This excitement about new cultures and their liquors of choice went through the mid-20th century. Think martinis at lunch and Manhattans. Unfortunately, it came to an end as the 60s and 70s were replaced with drug culture. 

Finally, in the 90s, there was a resurgence of ambitious bartenders who wished to reignite the classic art of cocktail making. They wanted to bring historical value and provide a cocktail background to the different beverages, showcasing quality standards to the different drinks.

Today, classic cocktails and even the more historical ones can be found more easily, but you probably won’t be able to order a lot of the drinks found in The Bartender’s Guide as they slowly faded out of style. 

However, the skill of bartending and mixology still stands, following in the footsteps of Professor Thomas. Now, where did each cocktail come from, and what are their meanings?

THE HISTORY OF THE DAIQUIRI
One of the most interesting in the history of cocktails, the history of the daiquiri cocktail, comes from an actual cocktail recipe card dating back to 1896 by someone named “Jennings Cox.” Cox, who lived and worked in Cuba and worked as an American engineer, ran out of gin one day while throwing a cocktail party. 

Rum being plentiful in Cuba provided a convenient and easy substitute to the punch he was serving. He served the rum with limes and brown sugar, and it turned out to be a huge hit amongst the crowd. Cox called it the “Daiquiri,” naming it after a port town nearby.

Many contemplate if Cox was really the first to invent the Daiquiri as rum, brown sugar, and limes are so plentiful in Cuba. Either way, he was the first to put pen to paper and record the drink. In 1909, Admiral Lucius Johnson brought the cocktail to America after visiting Cuba and falling in love with the drink. Throughout the years, the cocktail underwent many transformations and was eventually turned into a frozen drink. It became known popularly as a tropical drink to have by the pool around the 1940s and 1950s, and chain restaurants eventually adopted it with an instant daiquiri cocktail mix.

THE HISTORY OF THE MARGARITA COCKTAIL
It’s difficult to say who created the beloved drink known as the margarita, a classic Tequila sour. One story claims that the drink was developed in 1938 by a Mexican restaurant owner who mixed it for his showgirl girlfriend. 

Another story claims that Texas socialite Margaret Sames, also known as Margarita, mixed the drink at a house party in Mexico in 1948. While no one can agree who created the drink, we know that the drink evolved from The Daisy, which was a classic cocktail that included mixed alcohol, grenadine, and citrus juice served over ice.

The original recipe for the Tequila version of the Daisy included Tequila, lime juice, orange liqueur, and a splash of soda. The margarita even means “daisy” in Spanish. Today, the recipe is pretty similar, using Blanco Tequila and a salted rim.

THE HISTORY OF THE COSMOPOLITAN COCKTAIL
The cosmopolitan is interesting in the history of cocktails. Again, a tough drink to pinpoint the origin of, the cosmopolitan is well known for its blush pink, sweet-tart formula, and appearances in pop culture. It is said that the drink predates World War II and initially started as the Gimlet. 

A Gimlet was made because you really had to add something to gin or vodka because the alcohol was made so poorly. People later discovered Cointreau from France after World War II, and the drink became what is known as a Kamikaze. Finally, in the 80s, they added cranberry juice, and that is how we have the Cosmopolitan.

THE HISTORY OF THE PALOMA COCKTAIL
The Paloma cocktail is a bit of a mystery in terms of the origin of the drink and the history of cocktails. It is known to be the most popular tequila based drink in Mexico, and many believe it to be named after La Paloma, which was a popular folk song in the early 1860s.

It could also have been created by the famous Don Javier Corona, the owner of La Capilla in Tequila, Mexico. Either way, the Paloma is the perfect drink on a sunny day with tequila and freshly squeezed grapefruit juice mixed with lime and agave. 
""";

...and run the summary function:

In [7]:
var summaryResult = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input });
Console.WriteLine($"Summary:\n{Utils.WordWrap(summaryResult.GetValue<string>(), 90)}\n");

Summary:
Arrr, matey! Cocktails be a treasure trove of history, with origins in British punches and
the pioneering work of Jerry Thomas. From the Daiquiri to the Margarita to the
Cosmopolitan, these drinks be sailin' through time with tales of rum, tequila, and fancy
mixers. So raise yer glass, me hearties, and toast to the swashbucklin' bartenders who
keep the cocktail tradition alive!




The same function above can be executed with less code:

In [8]:
string skPrompt = """
{{$input}}

Create bullet points as a fun trivia guide to the content above. 
""";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = input });

Console.WriteLine(result);


- **Brief History of Cocktails**:
  - Cocktails were inspired by British punches in the 18th century.
  - The word "cocktail" was officially defined in 1806 as a mixture of sugar, water, and bitters.
  - Jerry Thomas, known as the "Professor," was a pioneering American bartender who wrote a comprehensive recipe book on cocktails.
  - Ice availability, thanks to Frederic Tudor, significantly impacted cocktail making.
  - Prohibition temporarily halted cocktail consumption but later sparked a resurgence in cocktail culture post-World Wars.

- **The Daiquiri**:
  - Jennings Cox recorded the first daiquiri recipe in 1896, using rum, limes, and sugar in Cuba.
  - Cox named the drink after a nearby port town, and it later gained popularity in America through Admiral Lucius Johnson.
  - The daiquiri evolved into a frozen beverage and became synonymous with tropical settings.

- **The Margarita**:
  - The origin of the margarita cocktail is disputed, with different stories attributing its crea

Here's one more example of how to write an inline Semantic Function that gives a TLDR for a piece of text.




In [9]:
string skPrompt = @"
{{$input}}

Give me the TLDR in 5 funny words.
";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = input });

Console.WriteLine(result);

Cocktails: History, laughs, and drinks!
